##Dynamic programming

In [1]:
def print_arr(arr):
  for i in arr:
    print(i)

In [2]:
#На весах
def dynamic_programming_weight(costs, weights, max_weight, debug = False):
  result_array = [[0 for _ in range(max_weight+1)] for _ in range(len(weights)+1)]
  results = []
  for i in range(len(weights)+1):
    for w in range(max_weight+1):
      if i*w == 0:
        result_array[i][w] = 0
      elif weights[i-1] <= w:
        result_array[i][w] = max(result_array[i-1][w], costs[i-1] + result_array[i-1][w-weights[i-1]])
      else:
        result_array[i][w] = result_array[i-1][w]
      #print(result_array) # Если посимвольно хотим изменения смотреть
    results.append(result_array[i])
  if debug: return result_array[len(weights)][max_weight], results 
  return result_array[len(weights)][max_weight]


##2-approx

In [3]:
def max_greed(costs, weights, max_weight):
  cur_cost = 0
  to_analysis = [(costs[i], weights[i]) for i in range(len(costs))]
  to_analysis.sort(reverse=True)
  cur_weight = 0
  for el in to_analysis:
    if cur_weight + el[1] <= max_weight:
      cur_weight += el[1]
      cur_cost += el[0]
  return cur_cost

def greedy_alg(costs, weights, max_weight):
  to_analysis = [(costs[i]/weights[i], costs[i], weights[i]) for i in range(len(costs))]
  to_analysis.sort(reverse=True)
  cur_weight = 0
  cur_cost = 0
  for el in to_analysis:
    if cur_weight + el[2] <= max_weight:
      cur_weight += el[2]
      cur_cost += el[1]
  return cur_cost

def approx(costs, weights, max_weight, debug = False):
  if debug: return max(max_greed(costs, weights, max_weight), greedy_alg(costs, weights, max_weight)), [max_greed(costs, weights, max_weight), greedy_alg(costs, weights, max_weight)]
  return max(max_greed(costs, weights, max_weight), greedy_alg(costs, weights, max_weight))

##PTAS

In [4]:
from itertools import combinations
def set_of_subsets(a, k):
  result = []
  for i in range(k+1):
    result.append([j for j in combinations(a,i)])
  res = []
  for el in result:
    res.append([list(ele) for ele in el])
  return res

def greedy_search(n, max_weight, costs, weights, M):
  X = []
  z_g = 0
  w = max_weight
  if len([M]) == 0: return z_g, X
  for i in M:
      w -= weights[i-1]
  for i in range(1, n+1):
    if i not in M and weights[i-1] <= w:
      z_g += costs[i-1]
      w -= weights[i-1]
      X.append(i)
  return z_g, X

def PTAS(n, max_weight, costs, weights, k, debug = False):
  z_h = 0
  X_h = []
  A = [i for i in range(1, n+1)]
  M = set_of_subsets(A, k)
  for i in M:
    for j in i:
      if len(j):
        curr_sum = sum(weights[h-1] for h in j)
      else:
        curr_sum = 0
      if curr_sum <= max_weight:
        z_g, X = greedy_search(n, max_weight, costs, weights, j)
        curr_price = 0
        for h in j:
          curr_price += costs[h-1]
        if z_g + curr_price > z_h:
          z_h = z_g + curr_price
          X_h.append(set(X + j))
  if debug: return z_h, X_h[-1], X_h
  return z_h, X_h[-1]


##Branches and bounds

In [28]:
import math
def relaxed_knapsack(costs, weights, max_weight, _taken):
  ans = [0 for i in range(len(costs))]
  current_weight = 0
  current_price = 0
  visited = []
  #preprocessing
  for i in _taken:
    current_weight += weights[i[0]]*i[1]
    current_price += costs[i[0]]*i[1]
    visited.append(i[0])
    ans[i[0]] = i[1]
  if current_weight > max_weight:
    return -1, current_weight, ans
  to_analysis = [(costs[i]/weights[i], i) for i in range(len(costs)) if i not in visited]
  to_analysis.sort(reverse=True)
  for i in to_analysis:
      if current_weight + weights[i[1]] <= max_weight:
          current_weight += weights[i[1]]
          current_price += costs[i[1]]
          ans[i[1]] = 1
      else:
          frac = (max_weight - current_weight) / weights[i[1]]
          current_price += costs[i[1]] * frac
          current_weight += weights[i[1]] * frac
          ans[i[1]] = frac
          break
  return current_price, current_weight, ans

def branch_and_bounds(costs, weights, max_weight, upper_bound, debug = False):
  upper_bound, upper_weight, taken = relaxed_knapsack(costs, weights, max_weight, [])
  max_bound = 0
  taken_ind = [taken]
  if all(isinstance(x, int) for x in taken):
      return upper_bound, taken
  taken_list = [taken]
  solution = []
  while (not all(isinstance(x, int) for x in taken)):
    use_euristic = False
    taken_left = taken_ind.copy()
    taken_right = taken_ind.copy()
    for i in range(len(costs)):
      if type(taken[i]) != int:
        taken_left.append((i, 0))
        taken_right.append((i, 1))
        break
    founded_bound_1, weight_1, taken_1 = relaxed_knapsack(costs, weights, max_weight, taken_left.copy())
    founded_bound_2, weight_2, taken_2 = relaxed_knapsack(costs, weights, max_weight, taken_right.copy())
    if weight_1 > max_weight and weight_2 > max_weight:
      break

    if founded_bound_1 > max_bound and all(isinstance(x, int) for x in taken_1):
      max_bound = founded_bound_1
      solution = taken_1
    if founded_bound_2 > max_bound and all(isinstance(x, int) for x in taken_2):
      max_bound = founded_bound_2
      solution = taken_2
    if founded_bound_1 > founded_bound_2 and weight_1 <= max_weight:
      taken = taken_1
      taken_ind = taken_left
    else:
      taken = taken_2
      taken_ind = taken_right
  if debug: return taken_list
  return max_bound, solution

##TESTS

In [29]:
from statistics import mean 
import pandas as pd
import time

test_cases = {"t_1":[165, 
                     [23, 31, 29, 44, 53, 38, 63, 85, 89, 82], 
                     [92, 57, 49, 68, 60, 43, 67, 84, 87, 72],
                     [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]],
              "t_2":[26,
                     [12, 7, 11, 8, 9],
                     [24, 13, 23, 15, 16],
                     [0, 1, 1, 1, 0]],
              "t_3":[190,
                     [56, 59, 80, 64, 75, 17], 
                     [50, 50, 64, 46, 50, 5],
                     [1, 1, 0, 0, 1, 0]],
              "t_4":[50,
                     [31, 10, 20, 19, 4, 3, 6],
                     [70, 20, 39, 37, 7, 5, 10],
                     [1, 0, 0, 1, 0, 0, 0]],
              "t_5":[104,
                     [25, 35, 45, 5, 25, 3, 2, 2],
                     [350, 400, 450, 20, 70, 8, 5, 5],
                     [1, 0, 1, 1, 1, 0, 1, 1]],
              "t_6":[170,
                     [41, 50, 49, 59, 55, 57, 60],
                     [442, 525, 511, 593, 546, 564, 617],
                     [0, 1, 0, 1, 0, 0, 1]],
              "t_7":[750,
                     [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120],
                     [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240],
                     [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]]
}

dynamic_t = []
approx_t  = []
ptas_t    = []
bandb_t   = []
ideal_t   = []

def PTAS_univers(costs, weights, max_weight, debug = False):
    return PTAS(len(costs), max_weight, costs, weights, len(weights)//2, debug)

def branch_and_bounds_univers(costs, weights, max_weight, debug = False):
    return branch_and_bounds(costs, weights, max_weight, 0, debug)

backpack_funcs =   {"Dynamic" : (dynamic_programming_weight, dynamic_t),
                    "Approx"  : (approx,                     approx_t),
                    "PTAS"    : (PTAS_univers,               ptas_t),
                    "B&B"     : (branch_and_bounds_univers,  bandb_t)
}

def timer(_func):
    start_timer = time.perf_counter()
    _func
    stop_timer = time.perf_counter()
    return stop_timer - start_timer

iters = 1

for test_case in test_cases:
    max_weight, weights, costs, answer = test_cases.get(test_case)
    for func_name in backpack_funcs:
        func, time_array = backpack_funcs.get(func_name)
        temp_array = [timer(func(costs, weights, max_weight)) for _ in range(iters)]
        time_array.append(f'sec={"{:.2e}".format(mean(temp_array))}')

data = pd.DataFrame(data=[i[1:][0] for i in backpack_funcs.values()],
            columns=test_cases.keys(),
            index=backpack_funcs.keys()
            ).T

display(data)

,Dynamic,Approx,PTAS,B&B
t_1,sec=3.94e-07,sec=1.50e-07,sec=3.44e-07,sec=2.00e-07
t_2,sec=1.73e-07,sec=1.84e-07,sec=1.69e-07,sec=2.24e-07
t_3,sec=1.89e-07,sec=1.57e-07,sec=2.02e-07,sec=2.13e-07
t_4,sec=1.64e-07,sec=1.50e-07,sec=2.34e-07,sec=2.24e-07
t_5,sec=1.93e-07,sec=1.51e-07,sec=2.21e-07,sec=2.01e-07
t_6,sec=1.76e-07,sec=1.43e-07,sec=2.40e-07,sec=1.89e-07
t_7,sec=3.43e-07,sec=1.53e-07,sec=5.68e-07,sec=2.83e-07


In [31]:
for test_case in test_cases:
    print(test_case)
    max_weight, weights, costs, answer = test_cases.get(test_case)
    for func_name in backpack_funcs:
        func = backpack_funcs.get(func_name)[0]
        print(f"{func_name}: {func(costs, weights, max_weight)}")
    print()


t_1
Dynamic: 309
Approx: 309
PTAS: (309, {1, 2, 3, 4, 6})
B&B: (309, [1, 1, 1, 1, 0, 1, 0, 0, 0, 0])

t_2
Dynamic: 51
Approx: 47
PTAS: (51, {2, 3, 4})
B&B: (47, [1, 0, 1, 0, 0])

t_3
Dynamic: 150
Approx: 146
PTAS: (150, {1, 2, 5})
B&B: (119, [0, 1, 1, 0, 0, 1])

t_4
Dynamic: 107
Approx: 107
PTAS: (107, {1, 4})
B&B: (105, [1, 1, 0, 0, 0, 1, 1])

t_5
Dynamic: 900
Approx: 888
PTAS: (900, {1, 3, 4, 5, 7, 8})
B&B: (888, [0, 1, 1, 1, 0, 1, 1, 1])

t_6
Dynamic: 1735
Approx: 1735
PTAS: (1735, {2, 4, 7})
B&B: (1706, [0, 1, 0, 0, 0, 1, 1])

t_7
Dynamic: 1458
Approx: 1441
PTAS: (1458, {1, 3, 5, 7, 8, 9, 14, 15})
B&B: (1446, [1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0])

